In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv('normie.csv')
df2 = pd.read_csv('dark_patterns.csv')

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics
from joblib import dump

from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm, tree
from sklearn.linear_model import SGDClassifier, LogisticRegression


df1 = pd.read_csv('normie.csv')
df2 = pd.read_csv('dark_patterns.csv')

df1 = df1[pd.notnull(df1["Pattern String"])]
df1 = df1[df1["classification"] == 0]
df1["classification"] = "Not Dark"
df1.drop_duplicates(subset="Pattern String")

df2 = df2[pd.notnull(df2["Pattern String"])]
df2["classification"] = "Dark"
col = ["Pattern String","Pattern Category","Pattern Type", "classification"]
df2 = df2[col]

df = pd.concat([df1, df2])

X_train, X_test, y_train, y_test = train_test_split(
    df['Pattern String'], df["classification"], random_state=42, test_size=.3)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

# Model creation

classifiers = []
accs = []
cms = []

classifiers.append(BernoulliNB())
classifiers.append(MultinomialNB())
classifiers.append(RandomForestClassifier())
classifiers.append(svm.SVC())
classifiers.append(tree.DecisionTreeClassifier())
classifiers.append(SGDClassifier())
classifiers.append(LogisticRegression())

for clf in classifiers:
    clf.fit(X_train_tfidf, y_train)
    y_pred = clf.predict(count_vect.transform(X_test))
    accs.append(metrics.accuracy_score(y_test, y_pred))
    cms.append(metrics.confusion_matrix(y_test, y_pred))

for i in range(len(classifiers)):
    clf = classifiers[i]
    clf.fit(X_train_tfidf, y_train)
    y_pred = clf.predict(count_vect.transform(X_test))
    acc = metrics.accuracy_score(y_test, y_pred)
    cm = metrics.confusion_matrix(y_test, y_pred)

    print(f"{clf} accuracy: {acc}")
    print(f"Confusion Matrix: {cm}")

    # Get pattern types and categories for misclassified samples
    misclassified_samples = X_test[y_test != y_pred]

    misclassified_categories = df.loc[misclassified_samples.index, 'Pattern Category']
    misclassified_types = df.loc[misclassified_samples.index, 'Pattern Type']

    print(f"Misclassified Pattern Categories: {misclassified_categories.tolist()}")
    print(f"Misclassified Pattern Types: {misclassified_types.tolist()}")
    print("\n")

df1

BernoulliNB() accuracy: 0.9686888454011742
Confusion Matrix: [[410  19]
 [ 13 580]]
Misclassified Pattern Categories: [nan, 'Scarcity', nan, 'Urgency', nan, 'Scarcity', nan, 'Misdirection', nan, 'Obstruction', nan, 'Misdirection', nan, 'Urgency', 'Urgency', nan, 'Misdirection', nan, 'Scarcity', nan, 'Urgency', nan, 'Scarcity', nan, 'Urgency', nan, 'Scarcity', nan, 'Urgency', nan, nan, 'Misdirection', nan, 'Social Proof', nan, 'Scarcity', nan, 'Misdirection', nan, 'Social Proof', nan, nan, 'Urgency', nan, 'Urgency', nan, 'Social Proof', nan, 'Urgency', nan, 'Misdirection', nan, 'Misdirection', nan, 'Scarcity', nan, 'Scarcity', nan, 'Social Proof', 'Misdirection']
Misclassified Pattern Types: [nan, 'Low-stock Message', nan, 'Countdown Timer', nan, 'High-demand Message', nan, 'Pressured Selling', nan, 'Hard to Cancel', nan, 'Confirmshaming', nan, 'Countdown Timer', 'Countdown Timer', nan, 'Pressured Selling', nan, 'Low-stock Message', nan, 'Countdown Timer', nan, 'High-demand Message', na

,Pattern String,classification
0,FREE SHIPPING ON ORDERS OVER $100!,Not Dark
1,SOME EXCLUSIONS APPLY - LEARN MORE,Not Dark
2,HAVE A QUESTION? - CONTACT US,Not Dark
3,WELCOME TO 034MOTORSPORT!,Not Dark
4,SHOP AUDISHOP VOLKSWAGENPERFORMANCE SOFTWARE03...,Not Dark
...,...,...
2693,Blog,Not Dark
2694,Join Our List Get,Not Dark
2696,©-2019 Barnaby Ltd dba Good Vibrations,Not Dark
2698,18 U.S.C. §2257 Recordkeeping Requirements Com...,Not Dark
